# Music Generation with LSTM

In [12]:
import glob
import numpy as np
import pickle
import mido
import random
from mido import MidiFile, Message, MidiTrack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from keras_self_attention import SeqSelfAttention

In [2]:
# LUT for array indices storing note bits
channels = [0,1,2,9]
chanToArr = [0,1,2,0,0,0,0,0,0,3]

def parse( input_midi ):
    i = 1               # counter for each print statement
    listo = []

    prevChan = 0        # keeps track of previous channel to detect channel changes
    chanIt = 0          # iterates to the next channel in the forced order 0->1->2->9
    arr = [[0]*128]*4    # 12 notes each for channels(0,1,2,9) to keep track of continued notes

    # insert MIDI file here
    mid = MidiFile(input_midi) 
    temp = mido.merge_tracks(mid.tracks)

    # processing MIDI file
    for msg in temp:
        # look for note changes
        if(msg.type=='note_on'):

            # check for a new channel
            if(msg.channel != prevChan): 
                chanIt = chanIt + 1
                if(chanIt == 4):
                    chanIt = chanIt -4
                    listo.append([0,0,0])
                    i=i+1

                # iterate chanIt to "catch up" to the next listed channel
                while( channels[chanIt] != msg.channel):

                    # continue notes if not turned off
                    for a in range(len(arr[chanToArr[ channels[chanIt] ]])):
                        if(arr[chanToArr[ channels[chanIt] ]][a]>0):
                            listo[ arr[chanToArr[ channels[chanIt] ]][a] - 1 ][2] = listo[ arr[chanToArr[ channels[chanIt] ]][a] - 1 ][2] + 1
                    chanIt = chanIt + 1
                    if(chanIt == 4):
                        chanIt = chanIt -4
                        listo.append([0,0,0])
                        i=i+1
                    
            # check if note_on event is on or off switch
            if(msg.velocity==0):
                noteSwitch = "off"
                arr[chanToArr[msg.channel]][msg.note] = 0
            else:
                noteSwitch = "on"
                arr[chanToArr[msg.channel]][msg.note] = i        
                # print new note
                listo.append([msg.note , msg.channel, 1])
                i=i+1

            # update prevChan for detection
            prevChan = msg.channel

    for t in range(len(listo)):
        listo[t] = str(listo[t][0]) + "." + str(listo[t][1]) + "." + str(listo[t][2])
        
    return listo

In [3]:
def get_notes():
    i=0
    notes = None
    for file in glob.glob("samples/large/*.mid"):

        print("Parsing %s" % file)        
        if(i==0):
            notes = parse(file)
            i=i+1
        else:
            notes = np.concatenate((notes, parse(file)))
    
    pickle.dump(notes, open('notes.p', 'wb'))
    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 16

    pitchnames = sorted(set(item for item in notes))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)
    
    return (network_input, network_output)

In [4]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    hidden_layers = 256
    dropout = 0.4
    
    """ Initializing model """
    model = Sequential()
    
    """ Adding LSTM Layers to Model """
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            ),
            input_shape=(network_input.shape[1], network_input.shape[2])
        )
    )
    model.add(SeqSelfAttention())
    model.add(LayerNormalization())
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            )
        )
    )
    model.add(SeqSelfAttention())
    model.add(LayerNormalization())
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout
            )
        )
    )
    model.add(LayerNormalization())
    
    """ Add other layers after LSTM """
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layers))
    # model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.0))

    return model

In [5]:
def train_network(load_notes=False, to_load_model=False, learning_rate=None):
    """ Train a Neural Network to generate music """
    notes = None
    model = None

    if load_notes:
        notes = pickle.load(open('notes.p', 'rb'))
    else:
        notes = get_notes()
    
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)

    if to_load_model:
        model = load_model('weights.hdf5', custom_objects={'SeqSelfAttention': SeqSelfAttention})
    else:
        model = create_network(network_input, n_vocab)

    if learning_rate != None:
        K.set_value(model.optimizer.learning_rate, learning_rate)

    checkpoint = ModelCheckpoint(
        'weights.hdf5',
        monitor='loss',
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]
    
    model.summary()

    model.fit(
        x=network_input,
        y=network_output,
        batch_size=32,
        epochs=3000,
        callbacks=callbacks_list
    )

In [6]:
train_network()

Parsing samples/large/091_DragonWarriorIV_40_41HornofBaron.mid
Parsing samples/large/090_DragonWarriorIII_06_07Victory.mid
Parsing samples/large/090_DragonWarriorIII_05_06BattleTheme.mid
Parsing samples/large/091_DragonWarriorIV_09_10MysteriousShrine.mid
Parsing samples/large/090_DragonWarriorIII_19_20Cursed.mid
Parsing samples/large/091_DragonWarriorIV_19_20Jackpot.mid
Parsing samples/large/090_DragonWarriorIII_30_31IntoTheLegend.mid
Parsing samples/large/091_DragonWarriorIV_03_04SolitaryWarrior.mid
Parsing samples/large/088_DragonWarrior_00_01Overture.mid
Parsing samples/large/091_DragonWarriorIV_27_28ExpandingtheSeaMap.mid
Parsing samples/large/091_DragonWarriorIV_39_40TheUnknownCastle.mid
Parsing samples/large/091_DragonWarriorIV_06_07ImportantItemDiscovery.mid
Parsing samples/large/091_DragonWarriorIV_05_06CaveofFear.mid
Parsing samples/large/091_DragonWarriorIV_37_38BalloonsFlight.mid
Parsing samples/large/090_DragonWarriorIII_03_04Rondo.mid
Parsing samples/large/090_DragonWarrio

1920/1920 [==============================] - 50s 26ms/step - loss: 5.0977
Epoch 2/3000
1920/1920 [==============================] - 50s 26ms/step - loss: 4.8426
Epoch 3/3000
1920/1920 [==============================] - 49s 26ms/step - loss: 4.7863
Epoch 4/3000
1920/1920 [==============================] - 50s 26ms/step - loss: 4.7539
Epoch 5/3000
1920/1920 [==============================] - 52s 27ms/step - loss: 4.7003
Epoch 6/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 4.6370
Epoch 7/3000
1920/1920 [==============================] - 51s 27ms/step - loss: 4.5967
Epoch 8/3000
1920/1920 [==============================] - 52s 27ms/step - loss: 4.5696
Epoch 9/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 4.5410
Epoch 10/3000
1920/1920 [==============================] - 50s 26ms/step - loss: 4.5146
Epoch 11/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 4.4889
Epoch 12/3000
1920/1920 [=============================

1920/1920 [==============================] - 53s 28ms/step - loss: 3.6605
Epoch 95/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.6487
Epoch 96/3000
1920/1920 [==============================] - 52s 27ms/step - loss: 3.6530
Epoch 97/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.6428
Epoch 98/3000
1920/1920 [==============================] - 52s 27ms/step - loss: 3.6432
Epoch 99/3000
1920/1920 [==============================] - 51s 27ms/step - loss: 3.6290
Epoch 100/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.6224
Epoch 101/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.6180
Epoch 102/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.6093
Epoch 103/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.6167
Epoch 104/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.5994
Epoch 105/3000
1920/1920 [===============

1920/1920 [==============================] - 53s 28ms/step - loss: 3.2748
Epoch 187/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.2823
Epoch 188/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.2768
Epoch 189/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.2845
Epoch 190/3000
1920/1920 [==============================] - 51s 27ms/step - loss: 3.2809
Epoch 191/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.2812
Epoch 192/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.2721
Epoch 193/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.2653
Epoch 194/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.2741
Epoch 195/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.2693
Epoch 196/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.2556
Epoch 197/3000
1920/1920 [==========

1920/1920 [==============================] - 53s 28ms/step - loss: 3.1085
Epoch 279/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0978
Epoch 280/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.1030
Epoch 281/3000
1920/1920 [==============================] - 51s 27ms/step - loss: 3.1075
Epoch 282/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0997
Epoch 283/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.1014
Epoch 284/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.1004
Epoch 285/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0955
Epoch 286/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0951
Epoch 287/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.1017
Epoch 288/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0984
Epoch 289/3000
1920/1920 [==========

1920/1920 [==============================] - 53s 28ms/step - loss: 3.0314
Epoch 371/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0314
Epoch 372/3000
1920/1920 [==============================] - 51s 27ms/step - loss: 3.0287
Epoch 373/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0219
Epoch 374/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0291
Epoch 375/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0404
Epoch 376/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0242
Epoch 377/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0279
Epoch 378/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0314
Epoch 379/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0225
Epoch 380/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0353
Epoch 381/3000
1920/1920 [==========

1920/1920 [==============================] - 53s 28ms/step - loss: 3.0250
Epoch 463/3000
1920/1920 [==============================] - 51s 27ms/step - loss: 3.0202
Epoch 464/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0345
Epoch 465/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0210
Epoch 466/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0277
Epoch 467/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0217
Epoch 468/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0163
Epoch 469/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0292
Epoch 470/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0353
Epoch 471/3000
1920/1920 [==============================] - 53s 28ms/step - loss: 3.0225
Epoch 472/3000
1920/1920 [==============================] - 53s 27ms/step - loss: 3.0371
Epoch 473/3000
1920/1920 [==========

KeyboardInterrupt: 

In [7]:
def prepare_sequences_prediction(notes, pitchnames, n_vocab):

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = 16
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [8]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []


    for note_index in range(1000):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)
        
        sum = 0
        i = 0
        
        for a in prediction[0]:
            sum = sum + a
        x = random.random() * sum
        
        for a in prediction[0]:
            x = x - a
            if(x < 0):
                break
            i = i+1

        index = i
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]
        
    return prediction_output

In [9]:
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif('rest' in pattern):
            new_rest = note.Rest(pattern)
            new_rest.offset = offset
            new_rest.storedInstrument = instrument.Piano() #???
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += convert_to_float(duration)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [10]:
def generate():
    notes = pickle.load(open('notes.p', 'rb'))
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_prediction(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    
    ### Add a line to load the weights here
    
    model.load_weights("weights.hdf5")
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)
    
def create_midi(prediction_output):
    i = 0
    for a in prediction_output:
        prediction_output[i] = [int(x) for x in a.split('.')]
        i=i+1
    
    t0notes = []
    t1notes = []
    t2notes = []
    t9notes = []
    
    for a in prediction_output:
        if(a[2] == 0):
            pass
        else:       
            if( a[1] == 0 ):
                t0notes.append([a[0],a[2]])
            elif( a[1] == 1 ):
                t1notes.append([a[0],a[2]])
            elif( a[1] == 2 ):
                t2notes.append([a[0],a[2]])
            elif( a[1] == 9 ):
                t9notes.append([a[0],a[2]])
                                    
#     print(t0notes)
#     print(t1notes)
#     print(t2notes)
#     print(t9notes)
    
    
    mid = MidiFile()
    mTrack = MidiTrack()
    track0 = MidiTrack()
    track1 = MidiTrack()
    track2 = MidiTrack()
    track9 = MidiTrack()
    mid.tracks.append(mTrack)
    mid.tracks.append(track0)
    mid.tracks.append(track1)
    mid.tracks.append(track2)
    mid.tracks.append(track9)

    track0.append(Message('program_change', channel=0, program=80, time=0))
    track1.append(Message('program_change', channel=1, program=81, time=0))
    track2.append(Message('program_change', channel=2, program=38, time=0))
    track9.append(Message('program_change', channel=9, program=121, time=0))

    for a in t0notes:
        track0.append(Message('note_on', channel=0, note=a[0], time=a[1]*100))
        track0.append(Message('note_on', channel=0, note=a[0], velocity=0))
    for a in t1notes:
        track1.append(Message('note_on', channel=1, note=a[0], time=a[1]*100))
        track1.append(Message('note_on', channel=1, note=a[0], velocity=0))
    for a in t2notes:
        track2.append(Message('note_on', channel=2, note=a[0], time=a[1]*100))
        track2.append(Message('note_on', channel=2, note=a[0], velocity=0))
    for a in t9notes:
        track9.append(Message('note_on', channel=9, note=a[0], time=a[1]*100))
        track9.append(Message('note_on', channel=9, note=a[0], velocity=0))
    mid.save('output.mid')

In [13]:
generate()